# Writing a quantum program 1: wiring up

Qualtran lets you write quantum programs and subroutines by composing lower-level subroutines, gates, and operations. We call these composable objects *bloqs* because they are the quantum building blocks of a complex algorithm. Composition is recursive: composing (lower-level) bloqs defines (higher-level) bloqs. 

In this tutorial, you will write a very simple quantum program by composing bloqs. The program will declare an input/output integer register named 'x' and use subroutines to negate the integer.

## Using `BloqBuilder`

Before we write our simple `negate` program, we'll write the simplest possible program:
let's create a program that takes an integer and does nothing to it. In this code snippet, we

 - import `BloqBuilder`. We'll use methods on this class to construct our program.
 - import some data types. Here, we import `QInt` which specifies a quantum integer. This is a quantum data type *class*. When writing a quantum program, we need to instantiate these into quantum data type *objects* by providing any *compile-time classical parameters* to the data type's constructor.
 - declare a *register* for our program. We use the `add_register` method. We provide a name for the register; and we provide the quantum data type. The method returns a handle to the declared register that we can use later. These handles are called *soquets*. They will always be instantiated and consumed by the framework&mdash;the quantum programmer should never concern themselves with the internals of a Python soquet object.
 - immediately finish the program. Since this first program does nothing, we immediately return. Our call to `bb.finalize` maps output register names to soquets.  

In [ ]:
# Quantum program 1
# This program does nothing
# 
# Registers:
#   x: an 8-bit signed quantum integer
from qualtran import BloqBuilder, QInt
import qualtran.testing as qlt_testing
from qualtran.drawing import show_bloq
# Start program construction
dtype = QInt(bitwidth=8, signed=True)

bb = BloqBuilder()

# Add input/output registers named 'x'

do_nothing = bb.finalize(x=x)

At a very basic level, we want the structure of our program to be valid. We can use the following check to do some basic assertions about the structure of the program.

In [ ]:
import qualtran.testing as qlt_testing
qlt_testing.assert_valid_cbloq(do_nothing)

We can also show a directed acyclic graph representation of the program, which is similarly simple.

In [ ]:
from qualtran.drawing import show_bloq
show_bloq(do_nothing)

## Negating an integer: pseudocode

In mathematics, there are infinitely many positive and negative integers. In engineering, we sadly must encode our integers into bits or qubits which have a finite range of values. In Qualtran, signed integers are encoded using twos-compliment

In [ ]:
print(QInt(8).to_bits( 1))
print(QInt(8).to_bits(-1))

It is beyond the scope of this tutorial, but take some time to convince yourself that the unary negation operation can be performed through

    -x = ~x + 1

where `(~x)` is the bitwise-not of each bit in the encoding of `x`.

To write our negation program, we'll use `BitwiseNot` to take the bitwise-not of x, and then `AddK` to add the classical compile-time constant `1`. Let's check the reference docs for these two operations.

## Negating an integer: code

If we look up the [documentation for `BitwiseNot`]() we can see what compile-time classical parameters and quantum registers we're expected to provide. 

> `BitwiseNot` <br/>
> **Parameters:** `dtype`: Data type of the input register `x`. <br/>
> **Registers:** `x`: A quantum register of type `self.dtype`.


Similarly, we can look up the documentation for `AddK`. It also takes a paremeter `dtype` and a register `x`. It additionally takes in the classical value we wish to add as parameter `k`.

We'll add two statements to our prior "do nothing" program to call these quantum subroutines.

In [ ]:
# Quantum program 2
# Negate a quantum integer
# 
# Registers:
#   x: an 8-bit signed quantum integer

from qualtran import BloqBuilder, QInt
from qualtran.bloqs.arithmetic import BitwiseNot, AddK
import qualtran.testing as qlt_testing
from qualtran.drawing import show_bloq

dtype = QInt(bitwidth=8, signed=True)

bb = BloqBuilder()
x = bb.add_register('x', dtype)

# -x = ~x + 1
x = bb.add(BitwiseNot(dtype), x=x)
x = bb.add(AddK(dtype, k=1), x=x)
# for x=bb.add(Addk)
negate = bb.finalize(x=x)

We always start by checking basic assertions about the structure of the program. If this assertion runs without error, the program is valid (although further tests should be performed to convince yourself it is *correct*).

In [ ]:
qlt_testing.assert_valid_cbloq(negate)

We can inspect the directed acyclic graph representation of the program. This contains the same information as is found in a traditional quantum circuit diagram.

In [ ]:
show_bloq(negate)

## Testing classical logic

Testing quantum programs is challenging in general because of the potentially exponential number of 'paths' connecting inputs and outputs. Our program only performs classical logic: if given a classical input (also known as a *computaitonal basis state*) it will return one and only one classical output. Although clearly one cannot achieve *quantum advantage* with only classical logic, a surprising number of *subroutines* in full quantum programs involve reversible-classical logic. In this context, the classical-reversible subroutines are called on superpositions of quantum data; but by factoring these parts into subroutines, we can write unit tests involving individual classical input/output states.

We exploit the simulability of bloqs that encode classical-reversible logic. Here, we call our program with `x=5` and note that the returned value is indeed negated. This can be extended to *fuzz test* a variety of input/output pairs for correctness.

In [ ]:
negate.call_classically(x=5)